In [1]:
using TaylorModels

In [77]:
#=
ord = 3

set_variables(Interval{Float64}, [:x, :y], order=2(ord+1))

x = TaylorN(Interval{Float64}, 1, order=ord)
y = TaylorN(Interval{Float64}, 2, order=ord)

b0 = 0 .. 1
ib0 = 0.0
xm = TaylorModelN(x, 0..0, b0, ib0)
=#

(These methods are adapted from RangeEnclosures.jl)

In [62]:
function enclose_IA(f::Function, dom::IntervalBox{N}, m::Int=1) where {N}
    val = [f(d...) for d in mince(dom, m)]
    lo = minimum(v.lo for v in val)
    hi = maximum(v.hi for v in val)
    return interval(lo, hi)
end

# multivariate
function enclose_TM(f::Function, dom::IntervalBox{N}, order::Int) where {N}
    x0 = mid(dom)
    set_variables(Float64, "x", order=2order, numvars=N)
    x = [TaylorModelN(i, order, IntervalBox(x0), dom) for i=1:N]
    return evaluate(f(x...), dom - x0)
end

@inline zeroBox(N) = IntervalBox(0..0, N)
@inline symBox(N) = IntervalBox(-1..1, N)

# normalized multivariate
function enclose_TM_normalized(f::Function, dom::IntervalBox{N}, order::Int) where {N}
    x0 = mid(dom)
    set_variables(Float64, "x", order=2order, numvars=N)

    zBoxN = zeroBox(N)
    sBoxN = symBox(N)
    x = [TaylorModelN(i, order, IntervalBox(x0), dom) for i=1:N]
    xnorm = [normalize_taylor(xi.pol, dom - x0, true) for xi in x]
    xnormTM = [TaylorModelN(xi_norm, 0..0, zBoxN, sBoxN) for xi_norm in xnorm]
    return evaluate(f(xnormTM...), sBoxN)
end

enclose_TM_normalized (generic function with 1 method)

In [63]:
f(x, y) = y  * exp(x * log(sin(x)))

dom = (0.9 .. 1.1) × (0.9 .. 1.1)

[0.899999, 1.10001] × [0.899999, 1.10001]

In [75]:
enclose_TM(f, dom, 4)

[0.713423, 0.969894]

In [76]:
enclose_TM_normalized(f, dom, 4)

[0.713423, 0.969894]

In [66]:
enclose_IA(f, dom, 1)

[0.687986, 0.991685]

In [67]:
enclose_IA(f, dom, 10)

[0.718902, 0.971337]